<a href="https://colab.research.google.com/github/karandeep7/Image-Classification-of-Plant-Diseases-Using-CNN/blob/main/plant_leaf_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fetching the data from source

###Download datasets from kaggle

In [ ]:
!kaggle datasets download -d rizwan123456789/potato-disease-leaf-datasetpld
!kaggle datasets download -d adilmubashirchaudhry/plant-village-dataset
!kaggle datasets download -d mexwell/crop-diseases-classification


Dataset URL: https://www.kaggle.com/datasets/rizwan123456789/potato-disease-leaf-datasetpld
License(s): DbCL-1.0
 56% 21.0M/37.4M [00:00<00:00, 95.9MB/s]
100% 37.4M/37.4M [00:00<00:00, 111MB/s] 
Dataset URL: https://www.kaggle.com/datasets/adilmubashirchaudhry/plant-village-dataset
License(s): CC-BY-NC-SA-4.0
 99% 979M/989M [00:15<00:00, 78.5MB/s]
100% 989M/989M [00:15<00:00, 68.4MB/s]
Dataset URL: https://www.kaggle.com/datasets/mexwell/crop-diseases-classification
License(s): other
100% 1.99G/2.00G [00:29<00:00, 75.1MB/s]
100% 2.00G/2.00G [00:29<00:00, 72.1MB/s]


###Unzip the datasets

In [ ]:
import zipfile
with zipfile.ZipFile('/content/crop-diseases-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/crop-diseases-classification')

with zipfile.ZipFile('/content/plant-village-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/plant-village-dataset')

with zipfile.ZipFile('/content/potato-disease-leaf-datasetpld.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/potato-disease-leaf-datasetpld')




#Preprocessing


###Import required libraries

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import pandas as pd
import os
from PIL import Image
import json
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from torch import tensor
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import vgg16
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

###Preprocessing functions for the dataloader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a standard size
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize

])

###Create a custom dataset class for Crop Diseases Classification dataset


In [ ]:
class CropDiseaseDataset(Dataset):
    def __init__(self, csv_file, img_dir, json_file, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

        # Load the JSON file with label mappings
        with open(json_file, 'r') as f:
            self.label_mapping = json.load(f)

        # Filter out rows where the image file is missing
        self.annotations = self.annotations[self.annotations['image_id'].apply(
            lambda x: os.path.exists(os.path.join(self.img_dir, x))
        )].reset_index(drop=True)
        self.classes = list(self.label_mapping.values())

    def __len__(self):
        return len(self.annotations)


    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])  # image_id
        image = Image.open(img_path)

        # Convert numerical label to disease name
        label_num = int(self.annotations.iloc[idx, 1])
        label = self.label_mapping[str(label_num)]

        # Convert the label to a numerical value if necessary
        label_index = label_num  # Assuming the label_num is the correct index for your class
        # Alternatively, you can map the label to an index if required

        if self.transform:
            image = self.transform(image)

        return image, tensor(label_index)  # Convert to tensor


###Create dataloader for Crop Diseases Classification Dataset

In [ ]:
# Define paths
csv_file = '/content/crop-diseases-classification/Data/train.csv'   # Update with actual path
img_dir = '/content/crop-diseases-classification/Data/train_images'        # Update with actual path
json_file = '/content/crop-diseases-classification/Data/label_num_to_disease_map.json' # Update with actual path

# Load full dataset
dataset_2 = CropDiseaseDataset(csv_file=csv_file, img_dir=img_dir, json_file=json_file, transform=transform)

# Split indices for train, validation, and test sets
train_indices, temp_indices = train_test_split(range(len(dataset_2)), test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Create subsets
train_dataset_2 = Subset(dataset_2, train_indices)
val_dataset_2 = Subset(dataset_2, val_indices)
test_dataset_2 = Subset(dataset_2, test_indices)

# Create DataLoaders
batch_size = 32

train_loader_2 = DataLoader(train_dataset_2, batch_size=batch_size, shuffle=True)
val_loader_2 = DataLoader(val_dataset_2, batch_size=batch_size, shuffle=True)
test_loader_2 = DataLoader(test_dataset_2, batch_size=batch_size, shuffle=True)
